# CLEVR-X 训练 & 提交流程（个人操作笔记）

从头训练 5 轮（epochs = 5） → 拿到一套干净的强模型 → 推理生成 `submission` → 上传看分数。  
按这套流程走，直接出结果。

---

## 第一步：从头训练 5 epoch（覆盖之前所有 LoRA）

在 Colab 里直接跑下面这个命令（在 Notebook 里记得前面加 `!`，这里我先写成纯命令行风格）：

```bash
python /content/clevr_comp/clevr_qwen2vl.py   --data_root /content/clevr_comp/custom_dataset   --output_dir /content/qwen2vl_clevr_lora   --do_train   --epochs 5   --batch_size 1   --grad_accum 4
```

**说明：**

 这一步会重新训练一套全新的 LoRA，之前 3epoch + 2epoch 那些就当历史了，直接覆盖。  
 5 epoch 对 Qwen2-VL-2B 来说刚好：  
   轮次不算少，模型有足够时间把 CLEVR-X 的分布学进去；  
   又不至于一上来就过拟合。  
 很多 CLEVR-X 的高分模型，训练轮次也就在 4～8 之间，5 epoch 完全合理。  

> 核心目的：先把一套「干净、统一训练过的强模型」整出来。

---

## 第二步：训练完成之后，立刻跑推理（`ensemble_size = 5`）

训练结束后，继续跑下面这条命令：

```bash
python /content/clevr_comp/clevr_qwen2vl.py   --data_root /content/clevr_comp/custom_dataset   --output_dir /content/qwen2vl_clevr_lora   --submission_path /content/submission_e5.csv   --ensemble_size 5   --gen_temperature 0.1   --gen_top_p 0.8
```

**说明：**

 对每一个 test 样本，模型会生成 **5 次输出**。  
 `ensemble_size = 5` = 多数表决：  
   answer 取 5 次里最稳定、最常出现的那个；  
   explanation 用这一次对应的自然语言推理句子，比赛规则也允许这么做。  
 `temperature = 0.1` + `top_p = 0.8`：  
   温度压低，让模型别乱飘，输出更稳定；  
   CLEVR-X 这种偏逻辑/计数题，低温基本等于「少犯迷糊」。  

执行完之后，会在 Colab 里得到一个文件：

```text
/content/submission_e5.csv
```

这个就是要上传的提交文件。

---

## 第三步：上传 `submission_e5.csv`，看 leaderboard 分数

把 `/content/submission_e5.csv` 上传到竞赛平台。

**预期：**

 分数一定会比之前的 `0.85452` 高；  
 正常情况下会落在 **0.90 ~ 0.92+** 这个区间；  
 如果 5 epoch + `ensemble_size = 5` 都跑得很稳，有希望摸到我想要的 **92% 左右**。  

---

## 训练完之后我只需要关心两件事

### 1. 看最后几轮的 loss 走势

在训练日志里，大概会是这种感觉（只是用来心里有数，不是硬性标准）：

 Epoch 1：loss 在 7.x ～ 6.x  
 Epoch 2：慢慢掉到 5.8～5.4  
 Epoch 3：差不多在 5.2 附近  
 Epoch 4：接近 5.0  
 Epoch 5：大概在 4.8～5.0 之间

如果 **最后两轮的 loss 都比较稳定**，没有突然爆炸或者开始明显往回涨，基本就可以当成一个可用的「强 baseline」。

### 2. 跑完推理 → 拿到 `submission_e5.csv` → 上传看分数

完整流程其实就两步：

1. 先把 5 epoch 训练跑完；  
2. 接着跑 ensemble 推理，把 `submission_e5.csv` 生成出来，然后上传。  

这套流程跑通之后，我就有一个比较靠谱的基线分数。后面要不要继续调参，看 leaderboard 上的结果再说。

---

##  后续如果想继续冲分，可以考虑的方向（备忘）

这些属于「第二阶段优化」，先不急着上，等 5epoch + ensemble=5 的分数出来再看：

 把 `ensemble_size` 从 5 提到 7，进一步提升答案稳定度；  
 微调采样策略（例如小范围调整 `temperature` / `top_p`）；  
 稍微优化 prompt，让 explanation 更集中在真正的推理步骤上；  
 把 epoch 适度加到 6～7，但要盯一下验证集表现，避免过拟合；  
 对训练集里的 explanation 做简单清洗，减少噪声、不相关描述。  

先把这版流程跑顺，再考虑堆料和花活。


In [ ]:
# Step 1: Train for 5 epochs (clean LoRA baseline)
!python /content/clevr_comp/clevr_qwen2vl.py \
  --data_root /content/clevr_comp/custom_dataset \
  --output_dir /content/qwen2vl_clevr_lora \
  --do_train \
  --epochs 5 \
  --batch_size 1 \
  --grad_accum 4


In [ ]:
# Step 2: Run inference with ensemble_size = 5
!python /content/clevr_comp/clevr_qwen2vl.py \
  --data_root /content/clevr_comp/custom_dataset \
  --output_dir /content/qwen2vl_clevr_lora \
  --submission_path /content/submission_e5.csv \
  --ensemble_size 5 \
  --gen_temperature 0.1 \
  --gen_top_p 0.8
